In [117]:
InputLots = str(input("What Lot IDs do you want to convert: "))
InputLots = InputLots.split()
lotIDs = "\""
for i in InputLots:
    lotIDs += "'" + i + "', "
lotIDs = lotIDs.rstrip(", ")
lotIDs += "\""

lotIDs = """ '8IQP5000', '8IQT02011', '8IQP02023' """
lotIDs

What Lot IDs do you want to convert: 8IQP5000 8IQT02011 8IQP02023


" '8IQP5000', '8IQT02011', '8IQP02023' "

In [116]:
import pandas as pd
import numpy as np
import pyodbc
con = pyodbc.connect('DSN=DB2v11')

daysback = 300

pullLotIDEight = """
-- sql for ELEC retrieval 
select distinct substr(last_fab_lot_id,1,9) as last_fab_lot_id_base, wafer_id
from DMIW_SYSTEMS.testedwafer
where substr(last_fab_lot_id,1,9) in (%s)
--and level in ('C5')
fetch first 1000000 rows only with ur 
;

"""%lotIDs
FabEight = pd.read_sql(pullLotIDEight,con)

lotdict = dict()
for i in FabEight.values.tolist():
    if i[0] not in lotdict.keys():
        lotdict[i[0]] = [i[1]]
    else:
        lotdict[i[0]].append(i[1])
print(lotdict)

DatabaseError: Execution failed on sql '
-- sql for ELEC retrieval 
select distinct substr(last_fab_lot_id,1,9) as last_fab_lot_id_base, wafer_id
from DMIW_SYSTEMS.testedwafer
where substr(last_fab_lot_id,1,9) in ("'8IQP5000', '8IQT02011', '8IQP02023'")
--and level in ('C5')
fetch first 1000000 rows only with ur 
;

': ('42S22', '[42S22] [IBM][CLI Driver][DB2/AIX64] SQL0206N  "\'8IQP5000\', \'8IQT02011\', \'8IQP02023\'" is not valid in the context where it is used.  SQLSTATE=42703\n (-206) (SQLExecDirectW)')

In [ ]:
pullLotIDTen = """
-- sql for MEASUREMENT retrieval 
select lot_Id as lot_Id, wafer_Id
 from DMIW.MeasuredRawFact measuredRawFact, 
	 DMIW_Systems.MeasuredParm measuredParm, 
	 (select measuredWaferKey, lot_Id, meas_Wafer_Id as wafer_Id, family_Code, lot_Grade, measurement_Date as date, proc_Level, meas_Pd_Id, meas_Eqp_Id 
	 from DMIW_Systems.MeasuredWafer measuredWafer 
	 where Measurement_date >= (current date - {} days) and Tech_id = '5H 14HP' and Meas_pd_id = 'MEACDRIE5CTP.1' and Wafer_id in ({}) 
	 group by measuredWaferKey, lot_Id, meas_Wafer_Id, family_Code, lot_Grade, measurement_Date, proc_Level, meas_Pd_Id, meas_Eqp_Id) measuredWafer 
 where measuredWafer.measuredWaferKey = measuredRawFact.measuredWaferKey and 
	 measuredParm.measuredParmKey = measuredRawFact.measuredParmKey and 
	 meas_Pd_Id <> '' and dcItem_Value is not null and 
	 ucase(dcItem_Name) not in ('XCOORD', 'YCOORD', 'XSITE', 'YSITE', 'FIELD X', 'FIELD Y', 'SITE X', 'SITE Y', 'SITEID', 'ROW', 'COLUMN') 
 	 and ( ucase(rtrim(meas_Pd_Id) || ' ' || dcItem_Name) like '%P23DIESITEVALUE%' )  
 group by lot_Id, wafer_Id, family_Code, lot_Grade,  dcItem_Name, proc_Level, meas_pd_Id, cntl_Lower_Limit, dcItem_Target, cntl_Upper_Limit, dcItem_Unit 
 order by 1, lot_Id, wafer_Id, dcItem_Name, proc_Level, meas_Pd_Id 
 fetch first 1000000 rows only with ur 
;
""".format(daysback, waferIDs)
FabTen = pd.read_sql(pullLotIDTen, con)

FabTen